In [1]:
import numpy as np
import pandas as pd

# %pip install -U pymoo

### Parámetros de entrada
Información que obtiene el modelo del sistema de información

In [2]:
# Demanda
DEMANDA = 60
# Costo por unidad de tiempo
CTIEMPO = 100
# Tiempo máximo definido
T_MAX = 360

# Parámetros de entrada
params_df = pd.read_excel('./data/' + 'nuevo_info_acopios.xlsx')

# Matriz de costos de transporte
costo_df = pd.read_excel('./data/' + 'costoTransporteCAi.xlsx', index_col=0)

# Matriz de tiempo de transporte
tiempo_df = pd.read_excel('./data/' + 'tiempoTransporteCAi.xlsx', index_col=0)

### Variables
Definición de variables para el modelo de optimización

In [3]:
N = params_df.shape[0]
capacidades = np.empty(N * 2, dtype=float)

for cap_i in range(0, N * 2, 2):
    capacidades[cap_i] = params_df['Stock'].iloc[cap_i // 2]
    capacidades[cap_i + 1] = params_df['Ppotencial'].iloc[cap_i // 2]
    
capacidades

array([10.   ,  0.5  , 12.   ,  0.625,  4.   , 14.   ,  8.   , 13.   ,
       18.   , 15.   ,  6.   , 14.   , 12.   , 13.   ,  2.   , 12.5  ,
       30.   , 15.   , 15.   , 18.   ])

# Función objetivo
$$
\begin{align*}
    \sum_{i=1 \quad i\neq p}^{N} &\big[ \, kCA_i \times Precio(CA_i) + kCA_i \times cTransp(CA_i) + Tiempo(CA_i) \times cTiempo \, \big] \, + \\
                                 &\big[ \, kCA_p \times Precio(CA_p) + Demanda \times cTransp(CA_p) + Tiempo(CA_p) \times cTiempo \, \big]
\end{align*}
$$

In [4]:
def f(x, idx_principal=0):
    delta = 0
    
    for i in range(0, N*2, 2):
        idx_acopio = i // 2

        if i == idx_principal or (x[i] == 0 and x[i+1] == 0):
            continue
            
        delta += get_delta(x, idx_acopio, idx_principal)

    delta += get_delta(x, idx_principal)
    
    return delta

def get_delta(x, idx_acopio, idx_principal=-1):
    kca = x[idx_acopio] + x[idx_acopio + 1]
    precio = params_df['Precio'].iloc[idx_acopio]
    talistam = params_df['TiempoAlistam'].iloc[idx_acopio]

    ' Si es el centro de acopio principal (idx_principal=-1) '
    if idx_principal < 0:
        ctransp = params_df['Ctransp'].iloc[idx_acopio]
        ttransp = params_df['TiempoTransp'].iloc[idx_acopio]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (DEMANDA * ctransp) + (tiempo * CTIEMPO)
    else:
        ctransp = costo_df.iloc[idx_acopio, idx_principal]
        ttransp = tiempo_df.iloc[idx_acopio, idx_principal]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (kca * ctransp) + (tiempo * CTIEMPO)

Sujeta a las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{N} kCA_i &= Demanda \\
    kCA_i &\leq Stock(CA_i) + Ppotencial(CA_i) &\therefore \, i=0,\cdots ,N \\
    TiempoAlistam(CA_i) &\leq TiempoMaxDefinido &\therefore \, i=0,\cdots ,N \\
    Tiempo(CA_i) &= TiempoAlistam(CA_i) + TiempoTransp(CA_i) \\
\end{align*}
$$

In [5]:
# Función para alterar el vector para que cumpla con la restricción de igualdad
def igualar(individuo, delta, diff):
    delta = np.squeeze(delta)
    
    # Reducir para igualar a la demanda
    if diff:
        acopios = list(np.nonzero(individuo)[0])

        while delta > 0 and len(acopios) > 0:
            idx = np.random.choice(acopios)
            acopios.remove(idx)

            if delta <= individuo[idx]:
                individuo[idx] -= delta
                delta = 0
            else:
                delta -= individuo[idx]
                individuo[idx] = 0
    # Aumentar para igualar a la demanda
    else:
        acopios = list(np.where(individuo == 0)[0])

        while delta > 0 and len(acopios) > 0:
            idx = np.random.choice(acopios)
            acopios.remove(idx)

            if delta <= capacidades[idx]:
                individuo[idx] = delta
                delta = 0
            else:
                individuo[idx] = capacidades[idx]
                delta -= capacidades[idx]

## pymoo

Iterar el modelo n veces según centros de acopio haya. La idea es asumir en cada iteración que el cacopio principal sea cada uno, desde i=0 hasta n.

Se evalúa el modelo tomando cada centro de acopio como el principal, y la ejecución da resultados para cada uno, luego de todas las soluciones se toma la mejor solución.

In [6]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(N*2)
xu = capacidades

' Definición del modelo de optimización '
class Queso(ElementwiseProblem):
    ca_principal = None
    
    def __init__(self, principal=None):
        self.ca_principal = principal
        
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=xl,
            xu=xu
        )
        
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(x, self.ca_principal)
        out['H'] = DEMANDA - np.sum(x)

model = Queso()

## Generación 
De nuevas posibles soluciones

In [7]:
from pymoo.core.sampling import Sampling

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        gen_matrix = np.zeros((n_samples, problem.n_var), dtype=float)

        for i in range(n_samples):
            while np.sum(gen_matrix[i]) < DEMANDA:
                idx = np.random.randint(problem.n_var)
                gen_matrix[i, idx] += np.random.choice((0, capacidades[idx]))

                if np.sum(gen_matrix[i]) > DEMANDA:
                    gen_matrix[i, idx] = gen_matrix[i, idx] - (np.sum(gen_matrix[i]) - DEMANDA)

        return gen_matrix

## Cruce
1. Para cada individuo en una muestra, se selecciona al azar otro individuo 
2. Se usa un índice (en medio) para separar el cruce de genes
3. Se crea el nuevo individuo 

In [8]:
from pymoo.core.crossover import Crossover



## Mutación
1. Tomar un índice (cacopio) aleatorio
2. **Si es cero:**
3. Cambiar a la cantidad máxima de la capacidad del centro de acopio
4. La cantidad que se asignó, restarla a lo largo de los demás centros de acopio para mantener la restricción de igualdad
5. **Sino:**
6. Cambiar a cero
7. La cantidad que se eliminó, sumarla en los demás centros de acopio para mantener la restricción de igualdad

In [9]:
from pymoo.core.mutation import Mutation

class ReassignMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            r = np.random.random()

            if r < 0.4:
                # Mutar capacidad de un individuo

                idx_mut = np.random.randint(X[i].shape)
                # False = Menor
                # True = Mayor
                diff = False

                if X[i][idx_mut] == 0:
                    delta = capacidades[idx_mut]
                    X[i][idx_mut] = capacidades[idx_mut]
                    diff = True

                else:
                    delta = X[i][idx_mut]
                    X[i][idx_mut] = 0

                igualar(X[i], delta, diff)

        return X

## Aplicación

In [10]:
def print_acopios(X):
    for i in range(0, N*2, 2):
        idx_ca = i // 2
        print(f'Centro de acopio {params_df['Id_CA'].iloc[idx_ca]}:')
        print(f'Stock: {params_df['Stock'].iloc[idx_ca]}, \t\t\tAsignado: {X[i]}')
        print(f'Potencial: {params_df['Ppotencial'].iloc[idx_ca]}, \tAsignado: {X[i+1]}\n')

In [22]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

class SinglePointCross(Crossover):
    def __init__(self):
        # Parents, Offspring
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        # n_parents, n_matings, n_var
        _, n_matings, n_var = X.shape

        T = np.zeros((1, n_matings, n_var))
        Y = np.full_like(T, None, dtype=float)

        for idx in range(n_matings):

            p1 = X[0, idx, : n_var//2]
            p2 = X[1, idx, n_var//2 : n_var]

            offspring = np.concatenate((p1, p2))

            if np.sum(offspring) > DEMANDA:
                delta = np.sum(offspring) - DEMANDA
                igualar(offspring, delta, True)
            else:
                delta = DEMANDA - np.sum(offspring)
                igualar(offspring, delta, False)

            for i in range(n_var):
                Y[0, idx, i] = offspring[i]

        return Y

algorithm = GA(
    pop_size=N*2,
    sampling=TopOrZeroSampling(),
    crossover=SinglePointCross(),
    mutation=ReassignMutation(),
    eliminate_duplicates=True
)

model.ca_principal = 8
res = minimize(model,
               algorithm,
               seed=1,
               verbose=False)
print_acopios(res.X)

KeyboardInterrupt: 

In [12]:
# for iteration in range(N):
#     model.ca_principal = iteration
# 
#     res = minimize(model,
#                    algorithm,
#                    seed=1,
#                    verbose=False)
# 
#     print(f'===========> Centro de acopio principal: {params_df['Id_CA'].iloc[iteration]}')
#     # print(f'F: {res.F}, \nX: {res.X}')
#     print_acopios(res.X)

In [17]:
# Implementación utilizando métodos predefinidos de pymoo como parámetros

# from pymoo.operators.sampling.rnd import IntegerRandomSampling
# from pymoo.operators.crossover.sbx import SBX
# from pymoo.operators.mutation.pm import PM
# from pymoo.operators.repair.rounding import RoundingRepair

# algorithm = GA(
#     # pop_size=200,
#     sampling=IntegerRandomSampling(),
#     crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
#     mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
#     eliminate_duplicates=True
# )
# 
# res = minimize(model,
#                algorithm,
#                seed=1,
#                verbose=False)
# 
# print(f'F: {res.F}, \nX: {res.X}')

## scipy

In [14]:
# c = tuple(idx for (_, idx) in bounds)
# Aeq = np.ones((1, len(c)))
# beq = 1

In [15]:
# from scipy.optimize import linprog
# # from scipy.optimize import differential_evolution
# 
# result = linprog(
#     c,
#     A_ub=Aeq,
#     b_ub=beq,
#     bounds=bounds,
#     method='highs',
#     integrality=1
# )
# print(result)